In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

from collections import Counter,defaultdict
from itertools import chain, combinations

import warnings
warnings.filterwarnings('ignore')

import imshow
from wordcloud import WordCloud
from PIL import Image

from itertools import chain
from collections import Counter
import pickle

import re
import sent2vec
import hnswlib
from gensim.models import Word2Vec
from gensim import corpora
from scipy.sparse import *
from scipy.sparse.linalg import svds

import math

from py_file.total_processing import konlpy_preprocessing
kp = konlpy_preprocessing()

In [2]:
song_meta = pd.read_json('data/song_meta.json')
song_meta['artists'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x))
train = pd.read_json('data/train.json')

display(song_meta.head(1))
display(train.head(1))

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artists
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,Various Artists


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


# 모델 불러오기

이전 파일 __3_1.Word2Vec_Knn_model_make.ipynb에서 생성한 word2vec와 knn모델들을 불러온다

In [3]:
w2v_model = Word2Vec.load('model/w2v.model')
p_tags = pickle.load(open('/home/ubuntu/workspace/jh/project/model/w2v_p_tags.pickle', 'rb'))
p_titles = pickle.load(open('/home/ubuntu/workspace/jh/project/model/w2v_p_titles.pickle', 'rb'))
p_new_tags = pickle.load(open('/home/ubuntu/workspace/jh/project/model/w2v_p_new_tags.pickle', 'rb'))

# 1. word2vec를 통해 연관 플레이리스트 뽑아오기

1. py파일로 만들어 놓은 자연어 전처리 함수에 한 개 이상의 단어를 입력
2. 단어들의 평균 vector값을 3개의 knn모델에 적용한다
3. 입력한 단어에 가장 유사한 플레이리스트를  2개씩 총 6개의 플레이리스트를 가져옴
4. 해당 플레이리스트의 노래들중 랜덤으로 30개 추천
5. k_n을 통해 가져올 플레이리스트의 수를 지정 할 수 있음


In [4]:
def w2v_recommendation(text, k_n=2):
    test_tag = kp.konlpy_preprocessing(text)

    test_tag = test_tag.split()
    tag_emb = []
    try:
        for txt in test_tag:
            tag_emb.append(w2v_model.wv.get_vector(txt))
    except ZeroDivisionError and KeyError:
        tag_emb.append(np.zeros(100))

    emb = sum(tag_emb)/len(tag_emb)

    if sum(emb) == 0:
        print('다시 입력해주세요!')
    else:
        title_labels, title_distances = p_titles.knn_query(emb, k = k_n, num_threads=8)
        tag_labels, tags_distances = p_tags.knn_query(emb, k = k_n, num_threads=8)
        new_tag_labels, new_tags_distances = p_new_tags.knn_query(emb, k = k_n, num_threads=8)

        reco_idx = list(chain.from_iterable(zip(title_labels.reshape(-1),tag_labels.reshape(-1),new_tag_labels.reshape(-1))))

        song_idx = list(set(np.concatenate(train.iloc[reco_idx]['songs'].tolist())))

        reco_songs = song_meta.iloc[song_idx,:][['song_name','artists']]

        return reco_songs.sample(30)

In [5]:
w2v_recommendation('아이유')

,song_name,artists
49564,사랑의 숲에서 길을 잃다 Part.2 (Feat. 영재 Of 4Men),제드
513270,Boo,아이유
339802,홀로 (Feat. 김나영),정키
147526,끝,임세준
97810,여자라서,아이유
387057,사랑과 행복 그리고 이별,이용
299260,날 안아줘,40
544096,골목을 돌면 (A-Live),거미
475555,Love Never Felt So Good,Michael Jackson
237808,귀여워 (With 권정열 Of 10cm),별
